In [1]:
import os, json
from PIL import Image
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models 

# ----
from data.dataset import FoodImageDataset

In [2]:
torch.cuda.is_available()

True

In [3]:
config = json.load(open('../config/config.json'))
mapping = json.load(open('../config/mapping.json'))
config

{'model': {'name': 'resnet50', 'num_classes': 10, 'input_size': 224},
 'dataset': {'train_path': '../../train', 'val_path': '../../val'},
 'training': {'batch_size': 32, 'epochs': 100, 'learning_rate': 0.001}}

In [5]:
foodimages = FoodImageDataset(config["dataset"]["train_path"])
foodimgaes_loader = DataLoader(foodimages, batch_size=64, shuffle=True)

for img, label in foodimgaes_loader:
    print(img.shape)
    print(label)


IndexError: list index out of range

In [ ]:
model = models.resnet50()